In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

In [43]:
books=pd.read_csv(r"C:\Users\bibhu\Desktop\Python\rcommended System\book_colaborative filter\Books.csv\Books.csv")
ratings=pd.read_csv(r"C:\Users\bibhu\Desktop\Python\rcommended System\book_colaborative filter\Ratings.csv\Ratings.csv")
users=pd.read_csv(r"C:\Users\bibhu\Desktop\Python\rcommended System\book_colaborative filter\Users.csv\Users.csv")

In [44]:
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [45]:
books['Image-URL-M']=books['Image-URL-M'].apply(lambda x: x.replace('http','https')) #to generate https link for deployment

In [46]:
ratings.head(2)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5


In [47]:
users.head(2)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


In [48]:
print(books.shape)
print(ratings.shape)
print(users.shape)

(271360, 8)
(1149780, 3)
(278858, 3)


In [49]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [50]:
users.isnull().sum()


User-ID          0
Location         0
Age         110762
dtype: int64

In [51]:
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [52]:
books.duplicated().sum()

0

In [53]:
ratings.duplicated().sum()

0

In [54]:
users.duplicated().sum()

0

**Popularity based recommendation system**

In [55]:
ratings_with_name=ratings.merge(books,on='ISBN')

In [56]:
ratings_with_name.head (2)

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,https://images.amazon.com/images/P/034545104X....,http://images.amazon.com/images/P/034545104X.0...
1,276726,0155061224,5,Rites of Passage,Judith Rae,2001,Heinle,http://images.amazon.com/images/P/0155061224.0...,https://images.amazon.com/images/P/0155061224....,http://images.amazon.com/images/P/0155061224.0...


In [57]:
popularity_df=ratings_with_name.groupby('Book-Title')['Book-Rating'].agg(['count','mean']).reset_index()

popularity_df=popularity_df.rename(columns={'count':'num_rating','mean':'avg_rating'})
popularity_df=popularity_df[popularity_df['num_rating']>250].sort_values('avg_rating',ascending=False).reset_index(drop=True)
popularity_df.head(2)

,Book-Title,num_rating,avg_rating
0,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804
1,Harry Potter and the Goblet of Fire (Book 4),387,5.824289


In [58]:
popularity_df=popularity_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_rating','avg_rating']]

In [59]:
popularity_df['Image-URL-M'][0]

'https://images.amazon.com/images/P/0439136350.01.MZZZZZZZ.jpg'

**Collaborative filtering**

In [60]:
#creating a dataset with users of rating more than 20 books and books with more than 50 rating
x=ratings_with_name.groupby('User-ID').count()['Book-Rating']>200
highRatedUser=x[x].index

In [61]:
filtered_rating=ratings_with_name[ratings_with_name['User-ID'].isin(highRatedUser)]
filtered_rating.head(2)

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
1150,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,https://images.amazon.com/images/P/002542730X....,http://images.amazon.com/images/P/002542730X.0...
1151,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...,https://images.amazon.com/images/P/0026217457....,http://images.amazon.com/images/P/0026217457.0...


In [62]:
y=filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books=y[y].index
famous_books

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Day Late and a Dollar Short', 'A Fine Balance',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='Book-Title', length=706)

In [63]:
filtered_rating=filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

In [64]:
filtered_rating.duplicated().sum()


0

In [65]:
filtered_rating['Book-Title'].nunique()

706

In [66]:
filtered_rating.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58586 entries, 1150 to 1029310
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   User-ID              58586 non-null  int64 
 1   ISBN                 58586 non-null  object
 2   Book-Rating          58586 non-null  int64 
 3   Book-Title           58586 non-null  object
 4   Book-Author          58586 non-null  object
 5   Year-Of-Publication  58586 non-null  object
 6   Publisher            58586 non-null  object
 7   Image-URL-S          58586 non-null  object
 8   Image-URL-M          58586 non-null  object
 9   Image-URL-L          58586 non-null  object
dtypes: int64(2), object(8)
memory usage: 4.9+ MB


In [67]:
pt=filtered_rating.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')
pt.fillna(0,inplace=True)
pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_score=cosine_similarity(pt)

In [69]:
def recommendation(book_name):
  book_index=np.where(pt.index==book_name)[0][0]
  similar_items=sorted(list(enumerate(similarity_score[book_index])),key=lambda x:x[1],reverse=True)[1:6]

  for i in similar_items:
    print(pt.index[i[0]]) #to print only book


In [70]:
#modified as per the web page

def recommendation(book_name):
    book_index=np.where(pt.index==book_name)[0][0]
    similar_items=sorted(list(enumerate(similarity_score[book_index])),key=lambda x:x[1],reverse=True)[1:6]
    data=[]
    for i in similar_items:
        item=[]  
        tem_df=books[books['Book-Title']==pt.index[i[0]]] # to print all the details
        item.extend(list(tem_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(tem_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(tem_df.drop_duplicates('Book-Title')['Image-URL-M'].values))

        data.append(item)
    return data

In [71]:
recommendation('1984')

[['Animal Farm',
  'George Orwell',
  'https://images.amazon.com/images/P/0451526341.01.MZZZZZZZ.jpg'],
 ["The Handmaid's Tale",
  'Margaret Atwood',
  'https://images.amazon.com/images/P/0449212602.01.MZZZZZZZ.jpg'],
 ['Brave New World',
  'Aldous Huxley',
  'https://images.amazon.com/images/P/0060809833.01.MZZZZZZZ.jpg'],
 ['The Vampire Lestat (Vampire Chronicles, Book II)',
  'ANNE RICE',
  'https://images.amazon.com/images/P/0345313860.01.MZZZZZZZ.jpg'],
 ['The Hours : A Novel',
  'Michael Cunningham',
  'https://images.amazon.com/images/P/0312243022.01.MZZZZZZZ.jpg']]

In [72]:
# books

In [73]:
import pickle
pickle.dump(popularity_df,open('popular.pkl','wb'))

In [74]:
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(similarity_score,open('similarity_score.pkl','wb'))
pickle.dump(books,open('books.pkl','wb'))


In [75]:
# from scipy.sparse import csr_matrix
# matrix=csr_matrix(pt.values)

In [76]:
# from sklearn.neighbors import NearestNeighbors
# model=NearestNeighbors(metric='cosine',algorithm='brute',n_neighbors=6)
# model.fit(matrix)

In [77]:
# dis,knear=model.kneighbors(pt.iloc[237,:].values.reshape(-1, 1),n_neighbors=6)

In [78]:
# pt.iloc[237,:].values.reshape(1,-1)